In [27]:
import time
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn import metrics

from libsvm.svmutil import *

In [28]:
data_x = np.load("data/linear_features/linear/k1p8nor3.npy")
data_y = np.load("data/linear_features/data_y.npy")
data_len = np.load("data/linear_features/data_len.npy")

In [3]:
data_x[0]

array([[-2.19668248e+02,  9.40386981e+02, -1.62594804e+03,
         1.45552731e+03, -7.10638789e+02,  1.75014826e+02,
        -1.58410534e+01,  1.74442281e+00, -7.74587535e-03],
       [-1.81074111e+02,  7.69450225e+02, -1.32374822e+03,
         1.18195569e+03, -5.75330654e+02,  1.39529950e+02,
        -1.22435813e+01,  1.99919682e+00, -1.09691091e-02],
       [ 2.82237092e+02, -1.19101915e+03,  2.02826060e+03,
        -1.77893787e+03,  8.40374450e+02, -1.95729308e+02,
         1.61783896e+01, -1.94652899e+00,  1.00902858e+00],
       [ 9.16036127e+01, -4.11879641e+02,  7.51180387e+02,
        -7.23228424e+02,  3.92605037e+02, -1.11054788e+02,
         1.19320700e+01, -1.69162204e+00,  1.00893680e+00]])

In [29]:
data_x = data_x.reshape(data_x.shape[0],-1)
print(data_x.shape)

(2432, 36)


In [23]:
data_x = np.append(data_x, np.array([data_len]).T, axis=1)

In [30]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.20, shuffle=True, random_state=12)

In [25]:
clf = svm.SVC(kernel='linear', C=1, class_weight='balanced').fit(x_train, y_train)
score = clf.score(x_test, y_test)
y_pred = clf.predict(x_test)
print("score:", score)
print("macro:", f1_score(y_test, y_pred, average='macro'))


In [6]:
import pickle
with open('data/linear_features/model/test_svm.pickle', 'wb') as f:
    pickle.dump(clf, f)

In [25]:
with open('data/linear_features/model/k1p8n3_svm.pickle', 'rb') as f:
    clf2 = pickle.load(f)
y_pred = clf2.predict(x_test)
print("macro:", f1_score(y_test, y_pred, average='macro'))

macro: 0.4361008662835577


In [18]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([ 147, 1798]))

In [33]:
prob = svm_problem(y_train, x_train, isKernel=True)
param = svm_parameter('-t 0 -w0 1798 -w1 147')
m = svm_train(prob, param)
p_label, p_acc, p_val = svm_predict(y_test, x_test, m)

print("macro:", f1_score(y_test, p_label, average='macro'))
print(metrics.confusion_matrix(y_test, p_label) / len(y_test))

Accuracy = 52.5667% (256/487) (classification)
macro: 0.41476394849785403
[[0.04517454 0.03696099]
 [0.43737166 0.48049281]]


In [35]:
import time

import numpy as np
import pandas as pd

from Features.dle import dle

from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, cross_val_score

# k_array = [1]
# power_array = [4]
# nor_array = [0]

k_array = [1]
power_array = [8]
nor_array = [3]
output_array = [4, 5, 10]

df = pd.read_csv("data/linear_features/cdhit80_data_seq_loc75_train.csv")
data_y = np.where(df["loc"].to_numpy() == "Cytosolic", 1, 0)

score_history = None

for k in k_array:
    for power in power_array:
        for nor in nor_array:
            print("k=%s, power=%s, nor=%s" % (k, power, nor))
            data_x, vocab = dle(df["Sequence"], k=k, power=power, normalized=nor)
            data_x = data_x.reshape(data_x.shape[0],-1)
            
            # np.save("data/linear_features/cdhit80_loc75_k" + str(k) + "_power" + str(power) + "_nor" + str(nor) + "_output" + str(output), np.append(np.array([data_y]).T, data_x, axis=1))
            
            x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.20, shuffle=True, random_state=12)

            print("x_train", x_train.shape)
            print("x_test", x_test.shape)
            print("y_test", np.unique(y_test, return_counts=True))

            clf = svm.SVC(kernel='rbf', C=1, class_weight='balanced').fit(x_train, y_train)
            
            score = clf.score(x_test, y_test)
            print("score:", score)
            macro = f1_score(y_test, p_label, average='macro')
            print("macro:", macro)
            if score_history is None:
                score_history = np.array([[k, power, nor, score, macro]])
            else:
                score_history = np.append(score_history, [[k, power, nor, score, macro]], axis=0)
print(score_history)
# np.save("data/linear_features/score_history/score_history", score_history)


k=1, power=8, nor=3
x_train (1945, 36)
x_test (487, 36)
y_test (array([0, 1]), array([ 40, 447]))
score: 0.5934291581108829
macro: 0.41476394849785403
[[1.         8.         3.         0.59342916 0.41476395]]
